In [61]:
import json
import os
import re
import yaml
from pathlib import Path
import pandas as pd

In [62]:
def load_names_data(names_file='names.yaml'):
    with open(names_file, 'r') as f:
        names_data = yaml.safe_load(f)

    name_lookup = {}

    for category, names in names_data.items():
        race, gender = category.split('_')

        for name in names:
            name_lookup[name] = (race.capitalize(), gender.capitalize())

    return name_lookup


def extract_negotiation_data(log_dir, name_lookup):
    """
    Extract buyer name, seller name, and final offer price from all game_state.json files
    in the given directory using the offer entries in game_state array.
    """
    results = []

    # Find all game_state.json files
    for subdir in sorted(os.listdir(log_dir)):
        subdir_path = os.path.join(log_dir, subdir)

        if not os.path.isdir(subdir_path):
            continue

        game_state_file = os.path.join(subdir_path, 'game_state.json')

        if not os.path.exists(game_state_file):
            continue

        try:
            with open(game_state_file, 'r') as f:
                data = json.load(f)

            # Extract settings from the first game state entry
            game_states = data.get('game_state', [])
            if not game_states:
                print(f"No game states found in {subdir}")
                continue

            settings = game_states[0].get('settings', {})
            buyer_name = settings.get('buyer_name', 'Unknown')
            seller_name = settings.get('seller_name', 'Unknown')

            # Get race and gender from name_lookup
            seller_race, seller_gender = name_lookup.get(seller_name, ('Unknown', 'Unknown'))
            buyer_race, buyer_gender = name_lookup.get(buyer_name, ('Unknown', 'Unknown'))

            # Check if deal was accepted
            end_tag = data.get('end_tag', None)
            final_offer = None

            if end_tag == 'ACCEPT':
                # Find the last offer in the game_state entries
                # Iterate through game states in reverse to find the most recent offer
                for state in reversed(game_states[1:]):  # Skip the START state
                    public_info = state.get('player_public_info_dict', {})
                    offer = public_info.get('offer', '')

                    # Check if there's an offer value
                    if offer and offer.strip():
                        try:
                            final_offer = int(offer)
                            break
                        except ValueError:
                            continue

            results.append({
                'log_id': subdir,
                'seller_name': seller_name,
                'seller_race': seller_race,
                'seller_gender': seller_gender,
                'buyer_name': buyer_name,
                'buyer_race': buyer_race,
                'buyer_gender': buyer_gender,
                'final_price': final_offer
            })

        except Exception as e:
            print(f"Error processing {game_state_file}: {e}")
            continue

    return results


In [63]:

name_lookup = load_names_data('names.yaml')

log_dir = '.logs/full_delegation'
results = extract_negotiation_data(log_dir, name_lookup)

res_df = pd.DataFrame.from_records(results)


In [64]:
res_df

,log_id,seller_name,seller_race,seller_gender,buyer_name,buyer_race,buyer_gender,final_price
0,1765414625517,Dustin Becker,White,M,Scott Becker,White,M,300000
1,1765414625541,Keyshawn Washington,Black,M,Todd Becker,White,M,301000
2,1765414625563,Matthew Becker,White,M,Tremayne Washington,Black,M,303000
3,1765414625577,DaShawn Washington,Black,M,DaQuan Washington,Black,M,310000
4,1765414853655,Jake Becker,White,M,Logan Becker,White,M,300000
...,...,...,...,...,...,...,...,...
87,1765426412992,Latrell Washington,Black,M,Keyshawn Washington,Black,M,453000
88,1765426413002,DaShawn Washington,Black,M,DaQuan Washington,Black,M,472000
89,1765426413011,DeAndre Washington,Black,M,DeAndre Washington,Black,M,505000
90,1765426413022,Latrell Washington,Black,M,DaShawn Washington,Black,M,514000


In [67]:
res_df.query('buyer_race == "Black"').final_price.describe()

count        43.000000
mean     477063.953488
std       59391.034001
min      302500.000000
25%      460000.000000
50%      480000.000000
75%      505000.000000
max      606000.000000
Name: final_price, dtype: float64

In [ ]:
res_df.query('buyer_race == "White"').final_price.describe()

count        44.000000
mean     463528.409091
std       64124.248396
min      300000.000000
25%      457500.000000
50%      467000.000000
75%      483125.000000
max      606000.000000
Name: final_price, dtype: float64